# War card game analysis
Assumptions:
- if there are n players, then there every player gets *(52 mod n)* cards and there are *(52 mod n) * n* cards in game
- non-playing cards are chosen from the lowest ones
- if there is a war and a player is out of cards (therefore can't continue the war) that player loses 

e.g. if there are n = 3 players:
- num_of_cards_per_player = 52 mod 3 = 17
- num_of_cards_in_game = 17 * 3 = 51
- deck = [2, 3, 3, 3, 3, 4, ...]

Steps:
1. Defining deck of cards and card values
2. Defining variables for analysis: numer of rounds ...
3. N simulations:
    - shuffling cards and assigning them to n players (1 stack for hand and 1 stack for side cards)
    - N loops:
        1. move - comparison card values
        2. if war, then: 1st move without comparison, 2nd move with comparison (check condition again - recurency)
        3. add cards from the move to winner's side cards stack
        4. if one of the players has all cards, then stop
4. Analysis ...

In [28]:
# libraries
import pandas as pd
import numpy as np
import ipywidgets as widgets

## Simulations

In [31]:
# Defining deck of cards and card values
"""
Jack = 11
Queen = 12
King = 13
Ace = 14
"""
card_values = list(range(2, 15))
deck = list(4 * card_values)
deck.sort()

print(card_values)
print(deck)

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
[2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6, 7, 7, 7, 7, 8, 8, 8, 8, 9, 9, 9, 9, 10, 10, 10, 10, 11, 11, 11, 11, 12, 12, 12, 12, 13, 13, 13, 13, 14, 14, 14, 14]


In [33]:
def check_hands(players):
    # if hand is empty
    for i, player in enumerate(players):
        if not player['hand']:
            if player['side']:
                # shuffle side cards and move them to the hand
                np.random.shuffle(player['side'])
                player['hand'], player['side'] = player['side'], []
            else:
                # delete player who lost (has no cards at all)
                players.pop(i)

In [35]:
def make_move(players, cards_for_side, print_cards=True, ignore_values=False):
    """
    players: list of players with hands and sides
    cards_for_side: list of cards accumulated in the round that will be added to winner's side cards
    ignore_values: used in war for cards that are reversed and their values are not taken into account
    return: True if game over, else False

    Note: works only for 2 players for now!
    """
    
    check_hands(players)
    if len(players) <= 1:
        return True
        
    # rozgrywka dla na razie 2 graczy
    player_1_card = players[0]['hand'].pop(0)
    player_2_card = players[1]['hand'].pop(0)
    if print_cards:
        print(player_1_card, end="\t")
        print(player_2_card)
    cards_for_side += [player_1_card, player_2_card]

    if ignore_values:
        print("(above cards are reversed)")
        make_move(players, cards_for_side, print_cards, False)
    else:
        if player_1_card > player_2_card:
            players[0]['side'] += cards_for_side
        elif player_1_card < player_2_card:
            players[1]['side'] += cards_for_side
        else:
            if print_cards:
                print("WAAAAR!!!!!")
            make_move(players, cards_for_side, print_cards, True)

In [37]:
def war_card_game_simulation(N, players_number):
    # variables for analysis
    rounds_in_each_simulation = []
    num_of_cards_in_game = 52 // players_number * players_number
    num_of_cards_per_player = num_of_cards_in_game // players_number
    
    # simulations
    for i in range(N):

        if i == 0: # print only first simulation
            print(f"------------------------------------------------ {i+1}. SIMULATION ------------------------------------------------")
        
        game_over = False
        iter = 0
        players = [ {'hand': [], 'side': []} for _ in range(players_number) ]
        
        # shuffling cards 
        shuffled_deck = deck.copy()
        shuffled_deck = deck[52-num_of_cards_in_game:]
        np.random.shuffle(shuffled_deck)

        # assign cards to players' hands
        for j, player in enumerate(players):
            player['hand'] = shuffled_deck[:num_of_cards_per_player]
            shuffled_deck = shuffled_deck[num_of_cards_per_player:]
            if i == 0:
                print(f"{iter}: player{j+1}\nhand: {player['hand']}\nside: {player['side']}")
        if i == 0:
            print()
            
        # game until someone wins
        max_iter = 10000
        print_cards = i == 0
        while not game_over:
            game_over = make_move(players, [], print_cards)
            iter += 1

            if i == 0:
                for j, player in enumerate(players):
                    print(f"{iter}: player{j+1}\nhand: {player['hand']}\nside:{player['side']}") 
                print()

            max_iter -= 1
            if max_iter <= 0:
                break
                
        rounds_in_each_simulation.append(iter-1)
        
    return rounds_in_each_simulation

In [39]:
rounds_in_100_simulations = war_card_game_simulation(N = 1, players_number = 2)


------------------------------------------------ 1. SIMULATION ------------------------------------------------
0: player1
hand: [11, 6, 9, 2, 8, 10, 10, 8, 6, 2, 3, 13, 7, 11, 4, 5, 12, 8, 3, 14, 13, 6, 2, 11, 11, 12]
side: []
0: player2
hand: [6, 5, 3, 3, 2, 4, 4, 4, 7, 9, 13, 13, 8, 10, 12, 9, 7, 12, 14, 10, 7, 5, 14, 9, 14, 5]
side: []

11	6
1: player1
hand: [6, 9, 2, 8, 10, 10, 8, 6, 2, 3, 13, 7, 11, 4, 5, 12, 8, 3, 14, 13, 6, 2, 11, 11, 12]
side:[11, 6]
1: player2
hand: [5, 3, 3, 2, 4, 4, 4, 7, 9, 13, 13, 8, 10, 12, 9, 7, 12, 14, 10, 7, 5, 14, 9, 14, 5]
side:[]

6	5
2: player1
hand: [9, 2, 8, 10, 10, 8, 6, 2, 3, 13, 7, 11, 4, 5, 12, 8, 3, 14, 13, 6, 2, 11, 11, 12]
side:[11, 6, 6, 5]
2: player2
hand: [3, 3, 2, 4, 4, 4, 7, 9, 13, 13, 8, 10, 12, 9, 7, 12, 14, 10, 7, 5, 14, 9, 14, 5]
side:[]

9	3
3: player1
hand: [2, 8, 10, 10, 8, 6, 2, 3, 13, 7, 11, 4, 5, 12, 8, 3, 14, 13, 6, 2, 11, 11, 12]
side:[11, 6, 6, 5, 9, 3]
3: player2
hand: [3, 2, 4, 4, 4, 7, 9, 13, 13, 8, 10, 12, 9, 7, 12, 

In [41]:
print(rounds_in_100_simulations)

[594]
